# TIF360 Project

Main source: https://www.kaggle.com/code/rmonge/predicting-molecule-properties-based-on-its-smiles/notebook

### Import packages

In [30]:
import os
import rdkit
from rdkit import Chem  # To extract information of the molecules
from rdkit.Chem import Draw  # To draw the molecules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import torch
from torch_geometric.loader import DataLoader
import torch_geometric.utils as utils
import networkx as nx
from torch.nn import Linear
from torch_geometric.nn import global_mean_pool, GraphConv, GATConv, GCNConv
import torch.nn.functional as F

from sklearn.metrics import r2_score

### Load data

In [31]:
df = pd.read_csv("../data/smiles_and_targets.csv")

# Investigate Neural Networks

## Graph Neural Networks

#### Convert data to graphs

In [32]:
# import packages
from rdkit.Chem import GetAdjacencyMatrix
import torch
from torch_geometric.data import Data
from torch.utils.data import DataLoader

In [33]:
def one_hot_encoding(x, permitted_list):
    if x not in permitted_list:
        x = permitted_list[-1]  # If the atom is not in the list, get "Unknown"
        
    binary_encoding = [int(boolean) for boolean in list(map(lambda s: x==s, permitted_list))]
    
    return binary_encoding    

Atom featurisation\
Currently generates ca. 80 node features

In [34]:
def get_atom_features(atom, use_chirality = True, hydrogens_implicit = True):
    # list of permitted atoms
    permitted_atom_list = ['C','N','O','S','F','Si','P','Cl','Br','Mg','Na','Ca',
                           'Fe','As','Al','I', 'B','V','K','Tl','Yb','Sb','Sn','Ag','Pd','Co',
                           'Se','Ti','Zn', 'Li','Ge','Cu','Au','Ni','Cd','In','Mn','Zr','Cr','Pt',
                           'Hg','Pb','Unknown']
    atom_type_enc = one_hot_encoding(str(atom.GetSymbol()), permitted_atom_list)
    
    n_heavy_neighbors = one_hot_encoding(int(atom.GetDegree()), [0,1,2,3,4,"MoreThanFour"])
    
    formal_charge_enc = one_hot_encoding(int(atom.GetFormalCharge()), [-3, -2, -1, 0, 1, 2, 3, 'Extreme'])
    
    hybridisation_type_enc = one_hot_encoding(str(atom.GetHybridization()), ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"])
    
    is_in_ring_enc = one_hot_encoding(int(atom.IsInRing()), [0, 1])
    
    is_aromatic_enc = one_hot_encoding(int(atom.GetIsAromatic()), [0, 1])
    
    atomic_mass_scaled = [float(atom.GetMass() - 10.812)/116.092] # (?) replace 10.812 with mean the and 116.092 with std
    
    vdw_radius_scaled = [float((Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5)/0.6)] # (?) replace 1.5 with mean the and 0.6 with std
    
    covalent_radius_scaled = [float((Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64)/0.76)] # (?) replace 0.64 with mean the and 0.76 with std
                              
    atom_feature_vector = atom_type_enc + n_heavy_neighbors + formal_charge_enc + hybridisation_type_enc + is_in_ring_enc + is_aromatic_enc + atomic_mass_scaled + vdw_radius_scaled + covalent_radius_scaled
    
    if use_chirality:
        chirality_type_enc = one_hot_encoding(str(atom.GetChiralTag()), ["CHI_UNSPECIFIED", "CHI_TETRAHEDRAL_CW", "CHI_TETRAHEDRAL_CCW", "CHI_OTHER"])
        atom_feature_vector += chirality_type_enc
        
    if hydrogens_implicit:
        n_hydrogens_enc = one_hot_encoding(int(atom.GetTotalNumHs()), [0, 1, 2, 3, 4, "MoreThanFour"])
        atom_feature_vector += n_hydrogens_enc
        
    return np.array(atom_feature_vector) 

Bond Featurisation\
Currently generates ca. 10 edge features

In [35]:
def get_bond_features(bond, use_stereochemistry=True):
    permitted_bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, 
                            Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]
    
    bond_type_enc = one_hot_encoding(str(bond.GetBondType()), permitted_bond_types)
    
    bond_is_conjugated_enc = [int(bond.GetIsConjugated())]
    
    bond_is_in_ring_enc = [int(bond.IsInRing())]
    
    bond_feature_vector = bond_type_enc + bond_is_conjugated_enc + bond_is_in_ring_enc
    
    if use_stereochemistry:
        stereo_type_enc = one_hot_encoding(str(bond.GetStereo()), ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"])
        bond_feature_vector += stereo_type_enc
        
    return np.array(bond_feature_vector)

Define function to generate dataset of labeled Pytorch Geometric Graphs

In [36]:
def create_graph_dataset_from_smiles(x_smiles, y):
    ## Inputs:
    # x_smiles = [smiles_1, smiles_2, ...], smiles representation of molecules
    # y = [y_1, y_2, ...] list of numerical labels for each smiles string, here chemical properties
    
    # Outputs:
    # dataset = [data_1, data_2, ...] list of torch_geometric.data.Data objects representing molecular graphs
    
    dataset = []
    
    for (smiles, y_val) in zip(x_smiles, y):
        # convert smiles to molecular object
        mol = Chem.MolFromSmiles(smiles)
        
        # get feature dimensions
        n_nodes = mol.GetNumAtoms()
        n_edges = 2*mol.GetNumBonds() # each bond is represented twice in the adjacency matrix
        n_node_features = len(get_atom_features(mol.GetAtomWithIdx(0)))
        if n_nodes > 1:
            n_edge_features = len(get_bond_features(mol.GetBondBetweenAtoms(0,1)))
        else:
            n_edge_features = 0  # for single atom molecules -> no edges
        
        # construct node feature matrix X 
        X = np.zeros((n_nodes, n_node_features))
        
        for atom in mol.GetAtoms():
            X[atom.GetIdx(), :] = get_atom_features(atom)
        
        X = torch.tensor(X, dtype=torch.float)
        
        # construct edge index array E, shape = (2, n_edges)
        (rows, cols) = np.nonzero(GetAdjacencyMatrix(mol))
        torch_rows = torch.tensor(rows.astype(np.int64)).to(torch.long)
        torch_cols = torch.tensor(cols.astype(np.int64)).to(torch.long)
        E = torch.stack([torch_rows, torch_cols], dim=0)
        
        # construct edge feature matrix EF
        EF = np.zeros((n_edges, n_edge_features))       # Note: generates zero matrix if n_edges = n_edge_features = 0
        for (k, (i,j)) in enumerate(zip(rows, cols)):
            EF[k] = get_bond_features(mol.GetBondBetweenAtoms(int(i),int(j)))
        EF = torch.tensor(EF, dtype=torch.float)
        
        # construct label/y tensor
        y_tensor = torch.tensor(np.array([y_val]), dtype=torch.float)
        
        # construct torch_geometric.data.Data object and append to dataset
        dataset.append(Data(x=X, edge_index=E, edge_attr=EF, y=y_tensor))
        
    return dataset
        

We use the above functions to create a dataset of molecular graphs from the smiles and labels corresponding to chemical properties

In [37]:
properties_names = ['A', 'B', 'C', 'mu', 'alfa', 'homo', 'lumo', 'gap', 'R²', 'zpve', 'U0', 'U', 'H', 'G', 'Cv']

x_smiles = df.smiles.values
y = df.loc[:, properties_names].values  # shape = (n_samples, n_properties)

dataset = create_graph_dataset_from_smiles(x_smiles, y[0:len(x_smiles), :])

In [38]:
print(len(dataset))

# Example entries
print(df.smiles.values[0])
print(dataset[0])
print(df.smiles.values[2])
print(dataset[2])
print(df.smiles.values[50])
print(dataset[50])


132820
C#CC#C
Data(x=[4, 81], edge_index=[2, 6], edge_attr=[6, 10], y=[1, 15])
N#CC#N
Data(x=[4, 81], edge_index=[2, 6], edge_attr=[6, 10], y=[1, 15])
CC1=CNC=C1
Data(x=[6, 81], edge_index=[2, 12], edge_attr=[12, 10], y=[1, 15])


Information of the graph dataset

In [39]:
print(f'Number of graphs (molecules): {len(dataset)}')
graph = dataset[50]
print('=================================================================================')
print(f'Properties of graph {50}, molecule smiles: {df.smiles.values[50]}')
print(f'Number of nodes: {dataset[50].x.shape[0]}')
print(f'Number of edges: {dataset[50].edge_index.shape[1]}')
print(f'Number of node features: {dataset[50].x.shape[1]}')
print(f'Number of edge features: {dataset[50].edge_attr.shape[1]}')
print(f'Number of properties: {dataset[50].y.shape[1]}')

Number of graphs (molecules): 132820
Properties of graph 50, molecule smiles: CC1=CNC=C1
Number of nodes: 6
Number of edges: 12
Number of node features: 81
Number of edge features: 10
Number of properties: 15


Filterout data with no edge features defined (Like ex: CH4) (These causes problems down the line)

In [40]:


indexes_to_delete = []
for item in range(0,len(dataset)):
    if dataset[item].edge_attr.shape[1] == 0:
        indexes_to_delete.append(item)
    else:
        pass

indexes_to_delete.sort()

print("Number of none edge feature molecules: ", len(indexes_to_delete))

print("Before: ", len(dataset))

for item in range(0,len(indexes_to_delete)):
    print("Molecule to delete: ", df.smiles.values[indexes_to_delete[item]])
    #del dataset[indexes_to_delete[item] - item] 
    dataset.pop((indexes_to_delete[item] - item)) # -item since all future data points will have its index reduced by 1 for each deleted previous data point
print("After: ", len(dataset))

Number of none edge feature molecules:  0
Before:  132820
After:  132820


Split data into train and test set

In [41]:
from torch_geometric.loader import DataLoader
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# split the dataset into test and validation:
num_samples = len(dataset)

# Want to divide data randomly
random.seed(42)
random_indexes = np.array(random.sample(range(num_samples), num_samples)) # random.sample ensures no duplicates

train_data = [dataset[index] for index in random_indexes[int(.2 * num_samples ) :]] # 80%
test_data = [dataset[index] for index in random_indexes[: int(.2 * num_samples )]] # 20%

print("Example train data target before scaling", train_data[0].y)
train_data_targets = [data.y for data in train_data]
train_data_targets = torch.concatenate(train_data_targets, axis=0)
test_data_targets = [data.y for data in test_data]
test_data_targets = torch.concatenate(test_data_targets, axis=0)

scaler = StandardScaler()
train_data_targets = scaler.fit_transform(train_data_targets)
test_data_targets = scaler.transform(test_data_targets)

# print("Example train data target", train_data_targets[0].reshape(1,-1).shape)
train_data_targets = torch.tensor(train_data_targets, dtype=torch.float)
test_data_targets = torch.tensor(test_data_targets, dtype=torch.float)

train_data = [Data(x=data.x, edge_index=data.edge_index, edge_attr=data.edge_attr, y=train_data_targets[index].reshape(1,-1)) for index, data in enumerate(train_data)]
test_data = [Data(x=data.x, edge_index=data.edge_index, edge_attr=data.edge_attr, y=test_data_targets[index].reshape(1,-1)) for index, data in enumerate(test_data)]
print("Example train data target after scaling:", train_data[0].y)

print("Total data size: ", len(dataset))
print("Train data size: ", len(train_data))
print("Test data size: ", len(test_data))

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

Example train data target before scaling tensor([[ 2.9693e+00,  1.9777e+00,  1.8423e+00,  1.2093e+00,  8.4060e+01,
         -1.8970e-01, -7.0400e-02,  1.1930e-01,  8.7339e+02,  1.5988e-01,
         -3.4861e+02, -3.4860e+02, -3.4860e+02, -3.4864e+02,  2.9189e+01]])
Example train data target after scaling: tensor([[-0.3350,  1.3300,  2.2841, -1.0264,  1.0990,  2.2975, -1.7445, -2.7966,
         -1.1527,  0.3357,  1.6155,  1.6155,  1.6155,  1.6156, -0.6067]])
Total data size:  132820
Train data size:  106256
Test data size:  26564


### Main GNN

#### Model for all targets at once

GNN function

In [42]:
data_labels = dataset[50].y.shape[1]
data_features = dataset[50].x.shape[1]

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(data_features, hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels)
        self.conv4 = GATConv(hidden_channels, hidden_channels)
        self.conv5 = GATConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, data_labels)

    def forward(self, x, edge_index, edge_attr, batch): 
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv3(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv4(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv5(x, edge_index, edge_attr)

        #Returns batch-wise graph-level-outputs by averaging node features across the node dimension, so that for a single graph G
        #its output is computed by
        x = global_mean_pool(x, batch) 
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(x)
        x = self.lin2(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(x)
        x = self.lin3(x)
        
 
        return x
    

Train GNN

In [43]:
model = GNN(hidden_channels=128) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=5e-4)
criterion = torch.nn.MSELoss()

def train(data_in):
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data_in.x, data_in.edge_index, data_in.edge_attr, data_in.batch)  # Perform a single forward pass.

      targets = data_in.y
      loss = criterion(out, targets) 

      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test(data):
      all_test_r2 = []
      counter = -1    
      for data_in in data:
            counter += 1
            model.eval()
            out = model(data_in.x, data_in.edge_index, data_in.edge_attr, data_in.batch)

            # Caculate R2
            r2_score_var = []
            for item in range(0,data_in.y.shape[1]):
                  if item == 0:
                        r2_score_var = r2_score(data_in.y[:,item].detach().numpy(), out[:,item].detach().numpy())
                  else:
                        r2_score_var = np.vstack((r2_score_var,(r2_score(data_in.y[:,item].detach().numpy(), out[:,item].detach().numpy()))))

            if counter == 0:
                  all_test_r2 = r2_score_var
            else:
                  all_test_r2 = np.hstack((all_test_r2,r2_score_var))

      average_test_r2 = np.sum(all_test_r2,axis=1) / all_test_r2.shape[1]

      return average_test_r2

# Vectors to append accuracy to:
Train_r2 = []
Test_r2 = []

# Calculate accuracy before training 
Train_r2.append(test(train_loader))
Test_r2.append(test(test_loader))
print("Initial training R2: ", Train_r2[0])
print("Initial test R2: ", Test_r2[0])

print_r2_option = True
for epoch in range(1, 31):
      average_loss = []
      for data in train_loader:
            loss = train(data)
            average_loss.append(loss)
      print(f'Epoch: {epoch:03d}, Loss: {(sum(average_loss)/len(average_loss)):.5f}')

      if print_r2_option:

            temp_train_r2 = test(train_loader)
            Train_r2.append(temp_train_r2)

            temp_test_r2 = test(test_loader)
            Test_r2.append(temp_test_r2)

            print(f'Average Train R2: {temp_train_r2}')
            print(f'Average Test R2: {temp_test_r2:}')

Initial training R2:  [-0.01865857 -0.01742911 -0.01774871 -0.02228407 -0.0201281  -0.02344883
 -0.01577503 -0.01612567 -0.0229795  -0.01822544 -0.01762917 -0.02177696
 -0.01757433 -0.02585516 -0.01675634]
Initial test R2:  [-0.0195153  -0.01639351 -0.0190408  -0.02689918 -0.01962406 -0.0210199
 -0.02546815 -0.02056046 -0.02444171 -0.01856565 -0.01779759 -0.02515549
 -0.01732174 -0.0235655  -0.01742934]
Epoch: 001, Loss: 0.53410
Average Train R2: [0.24880689 0.36706613 0.41893074 0.31687617 0.62654299 0.25176206
 0.69219754 0.64760428 0.59030507 0.81154387 0.62760868 0.6255647
 0.62785687 0.62762943 0.64958692]
Average Test R2: [0.265577   0.37901875 0.42993328 0.31407711 0.62735961 0.24115116
 0.69647084 0.64695372 0.59797961 0.81368156 0.64375137 0.64203859
 0.64418709 0.64391944 0.65213049]
Epoch: 002, Loss: 0.41042
Average Train R2: [0.24942971 0.52258773 0.58967103 0.359278   0.65592439 0.56680182
 0.77624317 0.71196212 0.71812648 0.85875174 0.68912926 0.68894879
 0.68946904 0.688

In [44]:
print("Final loss: ", (sum(average_loss)/len(average_loss)).detach().numpy())
print("Final training R2: ", Train_r2[-1])
print("Final test R2: ", Test_r2[-1])

Final loss:  0.26597893
Final training R2:  [0.59518922 0.68653812 0.7042399  0.41493737 0.78617766 0.75829724
 0.88722131 0.86579727 0.81688971 0.87568993 0.80171497 0.80172024
 0.80172021 0.80170896 0.82218064]
Final test R2:  [0.59661326 0.68558729 0.70455917 0.41200407 0.78386098 0.75963712
 0.88553335 0.86111517 0.81385668 0.87675062 0.80447125 0.80447628
 0.80447624 0.80446557 0.81996912]


#### Model for just one target

GNN function

In [45]:
data_labels = 1
data_features = dataset[50].x.shape[1]

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(data_features, hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels)
        self.conv4 = GATConv(hidden_channels, hidden_channels)
        self.conv5 = GATConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, data_labels)

    def forward(self, x, edge_index, edge_attr, batch): 
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv3(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv4(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv5(x, edge_index, edge_attr)

        #Returns batch-wise graph-level-outputs by averaging node features across the node dimension, so that for a single graph G
        #its output is computed by
        x = global_mean_pool(x, batch) 
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(x)
        x = self.lin2(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(x)
        x = self.lin3(x)
        
        return x

Train GNN

In [46]:
def train(data_in, target):
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data_in.x, data_in.edge_index, data_in.edge_attr, data_in.batch)
      targets = data_in.y[:,target].reshape(-1,1)
      
      #Alt 1
      loss = criterion(out, targets)   

      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test(data, target):
      all_test_r2 = []
      counter = -1    
      for data_in in data:
            counter += 1
            model.eval()
            out = model(data_in.x, data_in.edge_index, data_in.edge_attr, data_in.batch)
            targets = data_in.y[:,target].reshape(-1,1)
            
            # Caculate R2
            r2_score_var = r2_score(targets.detach().numpy(), out.detach().numpy())

            all_test_r2 .append(r2_score_var)

      average_test_r2 = np.sum(all_test_r2) / len(all_test_r2)

      return average_test_r2

num_targets = dataset[50].y.shape[1]
for target_index in range(num_targets):
      print("Target index: ", target_index)

      model = GNN(hidden_channels=64) 
      optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
      criterion = torch.nn.MSELoss()

      # Vectors to append accuracy to:
      Train_r2 = []
      Test_r2 = []

      # Calculate accuracy before training 
      Train_r2.append(test(train_loader, target_index))
      Test_r2.append(test(test_loader, target_index))
      print("Initial training R2: ", Train_r2[0])
      print("Initial test R2: ", Test_r2[0])

      print_r2_option = True
      for epoch in range(1, 21):
            average_loss = []
            for data in train_loader:
                  loss = train(data, target_index)
                  average_loss.append(loss)
            print(f'Epoch: {epoch:03d}, Loss: {(sum(average_loss)/len(average_loss)):.5f}')

            if print_r2_option:
                  temp_train_r2 = test(train_loader, target_index)
                  Train_r2.append(temp_train_r2)

                  temp_test_r2 = test(test_loader, target_index)
                  Test_r2.append(temp_test_r2)

                  # print(f'Average Train R2: {temp_train_r2}')
                  # print(f'Average Test R2: {temp_test_r2:}')

      print(f"Best training R2 for target {target_index}: {np.max(Train_r2)}")
      print(f"Best test R2 for target {target_index}: {np.max(Test_r2)}")

Target index:  0
Initial training R2:  -0.02350298644176647
Initial test R2:  -0.02277312422248588
Epoch: 001, Loss: 0.75478
Epoch: 002, Loss: 0.61201
Epoch: 003, Loss: 0.54524
Epoch: 004, Loss: 0.50621
Epoch: 005, Loss: 0.47388
Epoch: 006, Loss: 0.45273
Epoch: 007, Loss: 0.43445
Epoch: 008, Loss: 0.41947
Epoch: 009, Loss: 0.40344
Epoch: 010, Loss: 0.39646
Epoch: 011, Loss: 0.38529
Epoch: 012, Loss: 0.37788
Epoch: 013, Loss: 0.37201
Epoch: 014, Loss: 0.36414
Epoch: 015, Loss: 0.35948
Epoch: 016, Loss: 0.35402
Epoch: 017, Loss: 0.34916
Epoch: 018, Loss: 0.34466
Epoch: 019, Loss: 0.34472
Epoch: 020, Loss: 0.34064
Best training R2 for target 0: 0.6636045075432618
Best test R2 for target 0: 0.6633040708373179
Target index:  1
Initial training R2:  -0.019556148457033976
Initial test R2:  -0.020751837370101316
Epoch: 001, Loss: 0.63477
Epoch: 002, Loss: 0.51414
Epoch: 003, Loss: 0.45997
Epoch: 004, Loss: 0.41721
Epoch: 005, Loss: 0.39401
Epoch: 006, Loss: 0.37783
Epoch: 007, Loss: 0.36235
Ep